In [1]:
import pandas as pd
import os, time, datetime
os.chdir('..')
print(os.path.abspath('.'))

e:\Code\ML2020-ETA


## 测试数据的一些特征/分布

In [4]:
test_data = pd.read_csv('data/A_testData0531.csv', low_memory=False)
print(test_data['carrierName'].value_counts())

OIEQNT    30395
OYSCFP     5779
JCMFTA     4641
RWHZVZ     1823
UQCRKD     1297
BHSOUA      834
VRFMKJ      657
FXAJBJ       30
Name: carrierName, dtype: int64


In [5]:
print(test_data['vesselMMSI'].value_counts())

B8664636040    5572
C7827726749    4882
S7020352560    3810
X3806214427    2815
U3534410038    1921
               ... 
N9037513561      43
Q4197827697      43
M5147580600      38
G9393704581      37
U4075972506       7
Name: vesselMMSI, Length: 87, dtype: int64


In [6]:
print(test_data['TRANSPORT_TRACE'].value_counts())

CNYTN-MXZLO    25685
CNYTN-PAONX     5700
CNSHK-CLVAP     2900
CNYTN-ARENA     2833
CNSHK-MYTPP     1855
CNYTN-MATNG     1694
CNSHK-GRPIR      721
CNYTN-CAVAN      657
CNHKG-MXZLO      613
CNSHK-SGSIN      595
CNYTN-RTM        357
CNSHA-SGSIN      292
CNSHK-SIKOP      266
COBUN-HKHKG      245
HKHKG-FRFOS      223
CNYTN-NZAKL      165
CNSHK-ESALG      150
CNSHK-ZADUR      150
CNSHA-PAMIT      113
CNSHK-PKQCT      104
CNSHK-LBBEY       69
CNYTN-MTMLA       69
Name: TRANSPORT_TRACE, dtype: int64


## 训练数据里面和测试数据关联的信息

In [10]:
trace_set = test_data['TRANSPORT_TRACE'].value_counts().to_dict()

CHUNK_SIZE = 1024 * 1024
gps_record_reader = pd.read_csv('data/train0523.csv', header=None, low_memory=False, chunksize=CHUNK_SIZE)
for chunk in gps_record_reader:
    for trace in chunk[12]:
        if trace in trace_set:
            print(trace)
            del trace_set[trace]
    if len(trace_set) == 0:
        break
print(trace_set)

CNSHK-ZADUR
CNSHK-SGSIN
CNHKG-MXZLO
CNSHK-PKQCT
CNYTN-MATNG
CNSHK-CLVAP
CNYTN-NZAKL
CNYTN-MXZLO
CNYTN-PAONX
COBUN-HKHKG
CNYTN-RTM
CNSHK-SIKOP
CNYTN-MTMLA
CNYTN-ARENA
CNYTN-CAVAN
CNSHA-PAMIT
CNSHK-LBBEY
{'CNSHK-MYTPP': 1855, 'CNSHK-GRPIR': 721, 'CNSHA-SGSIN': 292, 'HKHKG-FRFOS': 223, 'CNSHK-ESALG': 150}


In [14]:
indirect_trace = [x.split('-') for x in ['CNSHK-MYTPP', 'CNSHK-GRPIR', 'CNSHA-SGSIN', 'HKHKG-FRFOS', 'CNSHK-ESALG']]
gps_record_reader = pd.read_csv('data/train0523.csv', header=None, low_memory=False, chunksize=CHUNK_SIZE)
for chunk in gps_record_reader:
    for trace in chunk[12]:
        if pd.isna(trace):
            continue
        s_trace = trace.split('-')
        for i, pair in enumerate(indirect_trace):
            if pair[0] in s_trace and pair[1] in s_trace:
                print(pair, s_trace)
                del indirect_trace[i]
    if indirect_trace == []:
        break
print(indirect_trace)

['HKHKG', 'FRFOS'] ['HKHKG', 'FRFOS', 'FRMRS', 'TNTUN']
['CNSHK', 'MYTPP'] ['CNSHK', 'MYTPP', 'MVMLE']
['CNSHK', 'ESALG'] ['CNSHK', 'ESALG', 'MRNKC']
['CNSHA', 'SGSIN'] ['CNSHA', 'SGSIN', 'OMSOH']
['CNSHK', 'GRPIR'] ['CNSHK', 'MYTPP', 'MYPKG', 'EGSUZ', 'GRPIR', 'ILASH', 'ILHFA', 'TRMER', 'CYFMG']
[]


## 筛选出一部分数据

缺失值真的不怎么好处理，但是这么筛下来还有3.19GB数据，足够了。

In [25]:
gps_record_reader = pd.read_csv('data/train0523.csv', header=None, low_memory=False, chunksize=CHUNK_SIZE)
trace_set = test_data['TRANSPORT_TRACE'].value_counts().to_dict()
trace_split_set = [x.split('-') for x in trace_set]

try:
    os.remove('data/filtered_data.csv')
except FileNotFoundError:
    pass
for chunk in gps_record_reader:
    selected_indexes = []
    for i, trace in enumerate(chunk[12]):
        if pd.isna(trace):
            continue
        if trace in trace_set:
            selected_indexes.append(i)
        else:
            s_trace = trace.split('-')
            for pair in trace_split_set:
                if pair[0] in s_trace and pair[1] in s_trace:
                    selected_indexes.append(i)
                    break
    filtered_df = chunk.loc[chunk.index[selected_indexes]]
    filtered_df.to_csv('data/filtered_data.csv', mode='a', header=False)
    count += CHUNK_SIZE
    print("processed one chunk")
print('finished data filtering')

processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk
processed one chunk


## 优化

1. 现在选的数据的周边数据很多都是可以用的，只不过有的需要补上trace